In [1]:
!pip install tensorflow-text==2.7.3
!pip install tensorflow==2.7.0
!pip install keras==2.7.0
!pip install cherrypicker
!pip install flask-ngrok
!pip install flask
!pip install pyngrok==4.1.1
!pip install -U flask-cors
!ngrok authtoken 1ovmnfSL2k9gaocbf0NvespmdiA_51HMB61MCc5Racn7NnB9
!wget https://bucket.asim.id/invisor2.h5

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
--2022-02-16 04:57:24--  https://bucket.asim.id/invisor2.h5
Resolving bucket.asim.id (bucket.asim.id)... 205.185.216.10, 205.185.216.42
Connecting to bucket.asim.id (bucket.asim.id)|205.185.216.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 434403912 (414M) [binary/octet-stream]
Saving to: ‘invisor2.h5.1’

invisor2.h5.1       100%[===================>] 414.28M   108MB/s    in 12s     

2022-02-16 04:57:37 (34.1 MB/s) - ‘invisor2.h5.1’ saved [434403912/434403912]



In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import tensorflow_hub as hub
import tensorflow_text as text
import json
import urllib.request
from cherrypicker import CherryPicker

In [3]:
model = tf.keras.models.load_model("invisor2.h5",custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS

app = Flask(__name__)
run_with_ngrok(app)
CORS(app)

@app.route('/')
def hello_world():
  return '<h1>Hello, World!</h1>'

@app.route("/get")
def get_prediction():
  stock = request.args["stock"]

  with urllib.request.urlopen("https://invisor.ml/api/allNews?stock="+stock) as url:
    Newsdata = json.loads(url.read().decode())
  
  picker = CherryPicker(Newsdata)
  flat = picker.flatten().get()
  textual_file = pd.DataFrame(flat)
  textual_file = textual_file.iloc[:15]

  with urllib.request.urlopen("https://invisor.ml/api/histData?stock="+stock) as url:
    num_json = json.loads(url.read().decode())
  num = pd.DataFrame(reversed(num_json))

  textual_file['combined'] = textual_file[textual_file.columns[1:27]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)

  textual_file['combined'].replace("[^a-zA-Z]"," ",regex=True, inplace=True)
  textual_file['combined']=textual_file['combined'].apply(lambda x : x.lower())

  textual_file=textual_file['combined']

  # num=num['close']
  # num = num.iloc[:15]

  scaler2=MinMaxScaler(feature_range=(0,1))
  num=num.values.reshape(-1,1)
  num_s=scaler2.fit_transform(num)

  newp=model.predict([textual_file,num_s])
  y_new_inverse = scaler2.inverse_transform(newp)
  res=y_new_inverse[0][0]
  prediction = json.dumps(res.item())

  return jsonify(prediction[:6])


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4f6d-34-71-199-73.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on /get [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functi